In [1]:
import pandas as pd
import numpy as np
import re
import pickle

In [2]:
df = pd.read_csv('data/data.csv', sep =';')

In [3]:
df['win_by'].value_counts()

Decision - Unanimous       1737
KO/TKO                     1647
Submission                 1083
Decision - Split            486
TKO - Doctor's Stoppage      70
Decision - Majority          56
Overturned                   35
DQ                           15
Could Not Continue           13
Other                         2
Name: win_by, dtype: int64

In [4]:
def winner (red,winner):
    if red == winner:
        return 1
    else:
        return 0

In [5]:
df['winner'] = df.apply(lambda x: winner(x['R_fighter'],x['Winner']),axis=1)

In [6]:
df['winner'].value_counts()
#whatever model gets selected needs to have an accuracy >67%

1    3470
0    1674
Name: winner, dtype: int64

In [7]:
#subset to decisions only
df = df[df['win_by'].str.contains("Decision")]

In [8]:
df['winner'].value_counts()

1    1456
0     823
Name: winner, dtype: int64

In [9]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR.', 'B_SIG_STR.',
       'R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_PASS', 'B_PASS', 'R_REV', 'B_REV', 'R_HEAD', 'B_HEAD', 'R_BODY',
       'B_BODY', 'R_LEG', 'B_LEG', 'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH',
       'B_CLINCH', 'R_GROUND', 'B_GROUND', 'win_by', 'last_round',
       'last_round_time', 'Format', 'Referee', 'date', 'location',
       'Fight_type', 'Winner', 'winner'],
      dtype='object')

In [10]:
#use the below to convert any string percents to actual percents
# df['col'] = df['col'].str.rstrip('%').astype('float') / 100.0
df['R_SIG_STR_pct'] = df['R_SIG_STR_pct'].str.rstrip('%').astype('float') / 100.0
df['B_SIG_STR_pct'] = df['B_SIG_STR_pct'].str.rstrip('%').astype('float') / 100.0
df['R_TD_pct'] = df['R_TD_pct'].str.rstrip('%').astype('float') / 100.0
df['B_TD_pct'] = df['B_TD_pct'].str.rstrip('%').astype('float') / 100.0


In [11]:
#subset to columns dealing with strikes, takedowns and submissions
X = pd.DataFrame()
X['strike_diff'] = df['R_SIG_STR_pct'] - df['B_SIG_STR_pct']
X['td_diff'] = df['R_TD_pct'] - df['B_TD_pct']
X['sub_diff'] = df['R_SUB_ATT'] - df['B_SUB_ATT']
X['pass_diff'] = df['R_PASS'] - df['B_PASS']

In [12]:
new1 = df["R_HEAD"].str.split("of", n = 1, expand = True) 
new2 = df["B_HEAD"].str.split("of", n = 1, expand = True) 
new3 = df["R_BODY"].str.split("of", n = 1, expand = True) 
new4 = df["B_BODY"].str.split("of", n = 1, expand = True) 
new5 = df["R_LEG"].str.split("of", n = 1, expand = True) 
new6 = df["B_LEG"].str.split("of", n = 1, expand = True) 
new7 = df["R_CLINCH"].str.split("of", n = 1, expand = True) 
new8 = df["B_CLINCH"].str.split("of", n = 1, expand = True) 
new11 = df["R_GROUND"].str.split("of", n = 1, expand = True) 
new12 = df["B_GROUND"].str.split("of", n = 1, expand = True) 

X['head_diff'] = new1[0].astype(float)/new1[1].astype(float) - new2[0].astype(float)/new2[1].astype(float)
X['body_diff'] = new3[0].astype(float)/new3[1].astype(float) - new4[0].astype(float)/new4[1].astype(float)
X['leg_dif'] = new5[0].astype(float)/new5[1].astype(float) - new6[0].astype(float)/new6[1].astype(float)
X['clinch_diff'] = new7[0].astype(float)/new7[1].astype(float) - new8[0].astype(float)/new8[1].astype(float)
X['ground_diff'] = new11[0].astype(float)/new11[1].astype(float) - new12[0].astype(float)/new12[1].astype(float)
X['kd_diff'] = df['R_KD'] - df['B_KD']


In [17]:
X = X.fillna(0)
X.describe()

,strike_diff,td_diff,sub_diff,pass_diff,head_diff,body_diff,leg_dif,clinch_diff,ground_diff,kd_diff
count,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000,2279.000000
mean,0.031435,0.068934,0.013602,0.594120,0.030515,0.027513,0.018602,0.016624,0.007799,-0.006143
std,0.169391,0.495057,1.280738,3.476849,0.181440,0.266089,0.267188,0.308276,0.270270,0.585872
min,-0.700000,-1.000000,-7.000000,-14.000000,-0.785714,-1.000000,-1.000000,-1.000000,-1.000000,-4.000000
25%,-0.080000,-0.270000,0.000000,-1.000000,-0.080367,-0.133333,-0.100000,-0.142857,0.000000,0.000000
50%,0.030000,0.000000,0.000000,0.000000,0.030780,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.140000,0.440000,0.000000,2.000000,0.140330,0.184772,0.138535,0.169478,0.000000,0.000000
max,0.800000,1.000000,10.000000,26.000000,0.739394,1.000000,1.000000,1.000000,1.000000,5.000000


In [20]:
tmp = X
tmp['winner'] = df['winner']
corr  = tmp.corr()
print(corr['winner'].sort_values(ascending=False))

winner         1.000000
head_diff      0.456355
strike_diff    0.447236
pass_diff      0.414691
td_diff        0.377615
body_diff      0.181708
kd_diff        0.173028
leg_dif        0.142092
clinch_diff    0.121812
ground_diff    0.023603
sub_diff       0.016548
Name: winner, dtype: float64


In [14]:
#initialize scale test and train data sets separetly to avoid leakage
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train_scaler=StandardScaler()
test_scaler=StandardScaler()

In [15]:
#set up dependant variable
y = df['winner'].copy()
len(y)

2279

# ADD A COMBINED DATA SET TO DETERMINE CORRELATION FOR FEATURE SELECTION!


In [16]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=21)

In [17]:
# Create a scaler 
train_scaler.fit(X_train)
test_scaler.fit(X_test)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [18]:
#archive the training scaler
filename = open('data/fitted_scaler.pkl', 'wb')
pickle.dump(train_scaler, filename)
filename.close()

In [19]:
# Now transform
X_train_scaled = train_scaler.transform(X_train)
X_test_scaled = test_scaler.transform(X_test)

In [20]:
# confirm that that did what we wanted
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)

In [21]:
# Pickle training data set for future graphical work
X_train.to_pickle('data/train.pkl') 

In [22]:
#random forest, logistic regression, Naive-Bayes? 
#Reminder: target accuracy is >67%
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import GaussianNB

log_clf = LogisticRegression() 
rnd_clf = RandomForestClassifier() 
gnb_clf = GaussianNB()

voting_clf = VotingClassifier(
estimators=[('lr', log_clf), ('rf', rnd_clf), ('gnb', gnb_clf)], voting='hard')
voting_clf.fit(X_train, y_train)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',...
                                        

In [23]:

from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, gnb_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.7894736842105263
RandomForestClassifier 0.7631578947368421
GaussianNB 0.7842105263157895
VotingClassifier 0.7842105263157895


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
